In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

/home/yu/miniconda3/envs/openmmlab/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
model_name = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

Error while downloading from https://cdn-lfs.hf.co/google/mt5-small/9daa76f0231b96162833ebfee26e886a781b72e0d95ad1a1826b9147a74a939a?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1729338695&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyOTMzODY5NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9nb29nbGUvbXQ1LXNtYWxsLzlkYWE3NmYwMjMxYjk2MTYyODMzZWJmZWUyNmU4ODZhNzgxYjcyZTBkOTVhZDFhMTgyNmI5MTQ3YTc0YTkzOWE%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIn1dfQ__&Signature=B%7EK33OIrzOcrRTW6G8Cjrf-LAjmCzMWH6Gv%7Ef7WUtrkN66ReNIAeIvSmsmnHTs2JDfFdPtwck0Cmp3jcgn8fYVHSZ9-vlKsYJfCucUj2HSWwrpW55m2R-8NIblKulFs6Yho--qiKtYl2L-r2iDM0j4yGS-cQzd6K-VpHVvatj8d%7EcW-y1ZcEZ1m58HYGgKcWGoJYZzSHSVvHpnG3xUoUXdyEjPc-2O-JwqJ04DALnw4WRs39yBIZFShaaB221uHj2Gmjnsfn9yBD-qz2Q1r0A6vIgJX%7EfioYnnQ5tfDsE8s7iNen2I2-C36EVzehDC

In [11]:
# from datasets import load_dataset
# dataset = load_dataset("wmt16", "ro-en")
from utils.rzbDataset import rzbDataset

k=0
train_dataset = rzbDataset(
    "data", k, mode="train")
val_dataset = rzbDataset(
    "data", k, mode="val")

print(train_dataset.data)

[INFO] Load 'data/train/fold_0.json' dataset...


100%|██████████| 1089/1089 [00:00<00:00, 246897.14it/s]


[INFO] Load 'data/val/fold_0.json' dataset...


100%|██████████| 136/136 [00:00<00:00, 126873.96it/s]

                                   original  \
0     训导处报告，训导处报告，三年二班周杰伦同学，你妈妈拿了两罐旺仔牛奶要给你。   
1     训导处报告，训导处报告，三年二班周杰伦同学，你妈妈拿了两罐旺仔牛奶要给你。   
2                  化学老师指着一罐锑向学生介绍道：“这是纯Sb。”   
3                  化学老师指着一罐锑向学生介绍道：“这是纯Sb。”   
4                  化学老师指着一罐锑向学生介绍道：“这是纯Sb。”   
...                                     ...   
2980                杨警官执行任务时不慎跌入粪坑，成为一名便衣警察   
2981                杨警官执行任务时不慎跌入粪坑，成为一名便衣警察   
2982                       功夫不负植物人他终于结出果实了！   
2983                       功夫不负植物人他终于结出果实了！   
2984                       功夫不负植物人他终于结出果实了！   

                                              annotated  
0                      这里借用了周杰伦的歌曲《三年二班》中的歌词，以及旺仔牛奶的广告。  
1                           三年二班是周杰伦的一首歌，同时这是旺仔牛奶广告中的句式  
2                                锑的化学元素号是Sb，而Sb也有骂人蠢的意思  
3     这里的“纯Sb”既是锑的化学符号，也是“纯”的发音的谐音，形成了一种双关语。化学老师在介绍锑...  
4                                sb原意是傻逼的意思，但恰好锑的元素式是sb  
...                                                 ...  
2980               便衣警察原指扮成路人

In [12]:
from datasets import Dataset
train_dataset = Dataset.from_dict(train_dataset.data)
val_dataset = Dataset.from_dict(val_dataset.data)
print(train_dataset, val_dataset)

Dataset({
    features: ['original', 'annotated'],
    num_rows: 2985
}) Dataset({
    features: ['original', 'annotated'],
    num_rows: 377
})


In [13]:
def tokenize_function(sample):
    model_input = tokenizer(sample["original"], max_length=32, truncation=True, padding="max_length")
    labels = tokenizer(sample["annotated"], max_length=32,
                       truncation=True, padding="max_length")
    model_input["labels"] = labels["input_ids"]
    return model_input


tokenized_train_datasets = train_dataset.map(tokenize_function, batched=True)
tokenized_val_datasets = val_dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 377/377 [00:00<00:00, 13804.52 examples/s]


In [14]:
print(tokenized_train_datasets, tokenized_val_datasets)

Dataset({
    features: ['original', 'annotated', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 2985
}) Dataset({
    features: ['original', 'annotated', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 377
})


In [15]:
tokenized_train_datasets.set_format(type="torch")
tokenized_val_datasets.set_format(type="torch")

In [16]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_train_datasets.shuffle(
    seed=42).select(range(100)), shuffle=True, batch_size=1)
eval_dataloader = DataLoader(tokenized_val_datasets.shuffle(
    seed=42).select(range(100)), batch_size=1)

In [17]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=1e-4)

In [18]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [20]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# device = torch.device("cpu")
model.to(device)

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
          

In [21]:
from tqdm import tqdm

progerss_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        # batch = {k: v.to(device) for k, v in batch.items()}
        batch = {
            "input_ids": batch["input_ids"].to(device),
            "attention_mask": batch["attention_mask"].to(device),
            "labels": batch["labels"].to(device),
        }
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progerss_bar.update(1)

100%|██████████| 300/300 [22:22<00:00,  4.33s/it]